In [15]:
from gradio_client import Client, handle_file   
# импорт contenxtmanager
from contextlib import contextmanager
import time
import os
from pydantic import BaseModel, Field
import csv
from rapidfuzz import process, fuzz
import re

from tqdm import tqdm
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import json
from pathlib import Path

In [2]:
class Timer:
    def __enter__(self):
        self.start = time.time()
        return self  # возвращаем self, чтобы получить доступ к elapsed_time

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time.time()
        self.elapsed = round(self.end - self.start,4)
        #print(f"Cell execution time: {self.elapsed:.4f} seconds")


In [3]:
with Timer() as t:
    time.sleep(1)

print(t.elapsed)

1.0001


In [18]:
DATA_BASE_DIR = Path("/home/jovyan/data")

In [58]:
with open(DATA_BASE_DIR / "test_dataset.json", "r") as f:
    test_data = json.load(f)

In [74]:
test_data[:1]

[{'image_path': 'names/930.jpg',
  'text': 'Молоко КРАСНАЯ ЦЕНА\n800мл\nмит.паст.2,5%'}]

In [51]:
client = Client("http://qwen_api:7860/")

Loaded as API: http://qwen_api:7860/ ✔


In [52]:
SYSTEM_PROMPT = '''
Вы — ассистент, который извлекает информацию с русских товарных этикеток на изображениях. В тексте могут встречаться названия на английском.
Выведи Только текст, без лишних слов. Никаких объяснений.
'''

In [66]:
def  predict(file_path, model_name = "Qwen2_5-7b"):    
    result = client.predict(
		system_prompt =SYSTEM_PROMPT,
		user_prompt="",
		image=handle_file(file_path),
        #image=None,
		selected_model=model_name,
		temperature=0.2,
		api_name="/predict"
	)
    return result

In [68]:
predict(DATA_BASE_DIR / "names/1.jpg")

'Вода святой источник 1.0л природная питьевая негаз.ПЭТ'

In [54]:


class PhraseCorrector:
    def __init__(self, words_file="words.csv",score_cutoff=70, min_len = 5):
        with open(words_file, "r", encoding="utf-8") as f:
            self.reference_words = [line.strip() for line in f]
        self.score_cutoff = score_cutoff
        self.min_len = min_len





    def correct_words_in_text(self,text):
        corrected_lines = []
        for line in text.strip().split('\n'):
            corrected_words = []
            words = re.split(r"[ \t\f\v.,!?;:()\"«»—–\-\/]+", line.upper().replace("Ё", "Е"))
            words = list(filter(None, words))  # убираем пустые строки, если они есть
        
            for word in words:
                if len(word) < self.min_len:
                    corrected_words.append(word)
                    continue
                # Ищем наиболее близкое слово из словаря
                match = process.extractOne(word, self.reference_words, scorer=fuzz.ratio, score_cutoff=self.score_cutoff)
                #print(match)
                corrected_word = match[0] if match else word
                corrected_words.append(corrected_word)
            corrected_lines.append(' '.join(corrected_words))
        return '\n'.join(corrected_lines)

In [55]:
phrase_corrector = PhraseCorrector(DATA_BASE_DIR / "words.txt")

In [76]:
class CSVRow(BaseModel):
    file_name: str
    text: str
    elapsed_time: float
    corrected_text: str
    elapsed_time_corrected: float
    original_text:str


In [86]:
def mass_experiment(model_name, limit=1, debug=False):
    with open(f"results_{model_name}.tsv", "w", encoding="utf-8") as f:
        writer = csv.writer(f, delimiter="\t", quoting=csv.QUOTE_MINIMAL)
        writer.writerow(CSVRow.model_fields.keys())
    
        for item in tqdm(test_data[:limit], desc="Process images", unit="image"):
            try:
                file_name = item["image_path"]
                original_text = item["text"]
                row = CSVRow(
                    file_name=file_name,
                    text="",
                    elapsed_time=0.0,
                    corrected_text="",
                    elapsed_time_corrected=0.0,
                    original_text=original_text
                )
    
                file_path = DATA_BASE_DIR / file_name
    
                # Чтение изображения
                if debug:
                    try:
                        img = mpimg.imread(file_path)
                        plt.imshow(img)
                        plt.axis('off')
                        plt.show()
                    except Exception as img_error:
                        print(f"[Ошибка изображения] {file_name}: {img_error}")
                        continue  # пропустить эту строку
        
                # Предсказание
                with Timer() as t:
                    result = predict(file_path, model_name)
                row.elapsed_time = t.elapsed
                row.text = result
    
                # Коррекция
                with Timer() as t:
                    corrected_result = phrase_corrector.correct_words_in_text(result)
                row.elapsed_time_corrected = t.elapsed
                row.corrected_text = corrected_result
    
                # Лог + запись
                if debug:
                    print(json.dumps(row.model_dump(), indent=4, ensure_ascii=False))
                writer.writerow(row.model_dump().values())
                f.flush()
    
            except Exception as e:
                print(f"[Общая ошибка] {item['image_path']}: {e}")
                continue


In [82]:
model_names = [
    "Qwen2_5-3b_finetuned",
    "Qwen2_5-7b_finetuned",
    "Qwen2-vl-7b_finetuned",
    "Qwen2_5-3b",
    "Qwen2_5-7b",
    "Qwen2-vl-7b"
]

In [87]:
for model_name in model_names:
    print(model_name)
    mass_experiment(model_name, -1)

Qwen2_5-3b_finetuned


Process images:   2%|▏         | 3/198 [00:07<06:18,  1.94s/image]

[Общая ошибка] names/577.jpg: The upstream Gradio app has raised an exception but has not enabled verbose error reporting. To enable, set show_error=True in launch().


Process images:  25%|██▌       | 50/198 [04:49<18:08,  7.35s/image]

[Общая ошибка] names/524.jpg: The upstream Gradio app has raised an exception but has not enabled verbose error reporting. To enable, set show_error=True in launch().


...

Process images:  71%|███████   | 140/198 [05:21<01:33,  1.62s/image]

[Общая ошибка] names/559.jpg: The upstream Gradio app has raised an exception but has not enabled verbose error reporting. To enable, set show_error=True in launch().


Process images:  86%|████████▋ | 171/198 [06:16<00:36,  1.37s/image]

[Общая ошибка] names/949.jpg: The upstream Gradio app has raised an exception but has not enabled verbose error reporting. To enable, set show_error=True in launch().


Process images: 100%|██████████| 198/198 [07:06<00:00,  2.16s/image]
